In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from typing import Union
from tqdm.notebook import tqdm

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch

In [2]:
!ls ../data/ml-1m

README      movies.csv  ratings.csv users.csv


In [3]:
import sys

sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/Project/recsysvenv/bin/python3.10'

## Constants

In [4]:
DATA_FOLDER = "../data/ml-1m/"

## Data

In [6]:
df_movies = pd.read_csv(
    DATA_FOLDER + "movies.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["movieId", "name", "genre"],
)
df_ratings = pd.read_csv(
    DATA_FOLDER + "ratings.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "movieId", "rating", "timestamp"],
)
df_users = pd.read_csv(
    DATA_FOLDER + "users.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "gender", "age", "occupation", "zip-code"],
)

In [7]:
# throw out movies that were not graded at all
df_movies = df_movies[df_movies.movieId.isin(df_ratings["movieId"].unique())]

In [8]:
## Encode usedId, movieId
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_movies["movieId"] = movie_encoder.fit_transform(df_movies["movieId"])
df_users["userId"] = user_encoder.fit_transform(df_users["userId"])

df_ratings["movieId"] = movie_encoder.transform(df_ratings["movieId"])
df_ratings["userId"] = user_encoder.transform(df_ratings["userId"])

#### Define test users for toy experiments

In [8]:
test_user_1 = (
    df_movies.loc[df_movies.genre.str.contains("Comedy")]
    .sample(60, random_state=777)
    .movieId.values
)

test_user_2 = (
    df_movies.loc[df_movies.genre.str.contains("Horror")]
    .sample(60, random_state=777)
    .movieId.values
)

test_user_3 = (
    df_movies.loc[df_movies.genre.str.contains("Sci-Fi")]
    .sample(60, random_state=777)
    .movieId.values
)


# test_user_1 = (df_movies
#                .loc[df_movies.genre == "Comedy"]
#                .sample(20, random_state=777).movieId.values)

# test_user_2 = (df_movies
#                .loc[df_movies.genre == "Horror"]
#                .sample(20, random_state=777).movieId.values)

# test_user_3 = (df_movies
#                .loc[df_movies.genre == "Sci-Fi"]
#                .sample(20, random_state=777).movieId.values)


test_user_1 = pd.DataFrame(test_user_1, columns=["movieId"]).assign(rating=5)
test_user_2 = pd.DataFrame(test_user_2, columns=["movieId"]).assign(rating=5)
test_user_3 = pd.DataFrame(test_user_3, columns=["movieId"]).assign(rating=5)

In [9]:
display(test_user_1.sample(5).merge(df_movies, on="movieId"))
display(test_user_2.sample(5).merge(df_movies, on="movieId"))
display(test_user_3.sample(5).merge(df_movies, on="movieId"))

,movieId,rating,name,genre
0,1561,5,Flubber (1997),Children's|Comedy|Fantasy
1,1351,5,That Darn Cat! (1997),Children's|Comedy|Mystery
2,1659,5,"Odd Couple II, The (1998)",Comedy
3,210,5,Billy Madison (1995),Comedy
4,3535,5,On Our Merry Way (1948),Comedy|Drama


,movieId,rating,name,genre
0,246,5,Interview with the Vampire (1994),Drama|Horror
1,3326,5,"Crow: Salvation, The (2000)",Action|Horror
2,1940,5,Cujo (1983),Horror|Thriller
3,1288,5,Jaws (1975),Action|Horror
4,2196,5,Psycho (1998),Crime|Horror|Thriller


,movieId,rating,name,genre
0,2517,5,Deep Blue Sea (1999),Action|Sci-Fi|Thriller
1,3045,5,Forever Young (1992),Adventure|Romance|Sci-Fi
2,2263,5,"Fly II, The (1989)",Horror|Sci-Fi
3,490,5,No Escape (1994),Action|Sci-Fi
4,3503,5,Titan A.E. (2000),Adventure|Animation|Sci-Fi


## Models

### SVD

Нужно убрать из рекоммендаций фильмы, которые он уже смотрел

In [10]:
class SVD:
    def __init__(self, random_seed: int = 777):
        self.random_seed = random_seed

    def fit(self, df: pd.DataFrame, k: int = 50):
        """
        * df: DataFrame with ratings
        * k: number of latent features
        """
        self.max_movieId = df.movieId.values.max()
        data_matrix = csr_matrix(
            (df.rating.values.astype("f8"), (df.userId.values, df.movieId.values))
        )
        _, _, self.Vt = svds(
            data_matrix,
            k=k,
            return_singular_vectors="vh",
            random_state=self.random_seed,
        )

    def predict(self, df: pd.DataFrame, k_best: int = 100) -> pd.DataFrame:
        """
        * k_best: return k best reccomendations for user. Default is None which returns all the ratings.
        ___
        Returns: DataFrame of ratings sorted in descending order.

        """
        a0 = (
            pd.DataFrame(np.arange(self.max_movieId + 1), columns=["movieId"])
            .merge(df[["movieId", "rating"]], on="movieId", how="left")
            .fillna(0)
            .loc[:, "rating"]
            .values
        )
        pred_ratings = self.Vt.T @ (self.Vt @ a0)
        res_df = (
            pd.DataFrame(np.arange(self.max_movieId + 1), columns=["movieId"])
            .assign(rating=pred_ratings)
            .sort_values(by="rating", ascending=False)
            .reset_index(drop=True)
        )

        if k_best:
            res_df = res_df.iloc[:k_best]

        return res_df

In [11]:
svd = SVD()
svd.fit(df_ratings, k=50)

In [16]:
display(test_user_1.sample(5).merge(df_movies, on="movieId"))
display(svd.predict(test_user_1, k_best=5).merge(df_movies, on="movieId"))

,movieId,rating,name,genre
0,2384,5,Never Been Kissed (1999),Comedy|Romance
1,2542,5,"Little Shop of Horrors, The (1960)",Comedy|Horror
2,2599,5,"Christmas Story, A (1983)",Comedy|Drama
3,138,5,"Brothers McMullen, The (1995)",Comedy
4,120,5,Flirting With Disaster (1996),Comedy


,movieId,rating,name,genre
0,3351,0.675054,Mr. Mom (1983),Comedy|Drama
1,2131,0.668345,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama
2,2990,0.623566,Fast Times at Ridgemont High (1982),Comedy
3,1963,0.616907,Sixteen Candles (1984),Comedy
4,3651,0.614373,Almost Famous (2000),Comedy|Drama


In [13]:
display(test_user_2.sample(5).merge(df_movies, on="movieId"))
display(svd.predict(test_user_2, k_best=5).merge(df_movies, on="movieId"))

,movieId,rating,name,genre
0,1793,5,Freddy's Dead: The Final Nightmare (1991),Horror
1,2511,5,Ghostbusters (1984),Comedy|Horror
2,214,5,Castle Freak (1995),Horror
3,3588,5,"Brain That Wouldn't Die, The (1962)",Horror|Sci-Fi
4,1570,5,Scream 2 (1997),Horror|Thriller


,movieId,rating,name,genre
0,1166,1.933450,"Shining, The (1980)",Horror
1,1288,1.923832,Jaws (1975),Action|Horror
2,1802,1.850637,Halloween (1978),Horror
3,1817,1.846517,"Exorcist, The (1973)",Horror
4,1305,1.772426,Scream (1996),Horror|Thriller


In [14]:
display(test_user_3.sample(5).merge(df_movies, on="movieId"))
display(svd.predict(test_user_3, k_best=5).merge(df_movies, on="movieId"))

,movieId,rating,name,genre
0,1913,5,"Rocketeer, The (1991)",Action|Adventure|Sci-Fi
1,3687,5,Kronos (1957),Sci-Fi
2,1682,5,Species II (1998),Horror|Sci-Fi
3,3458,5,"Brother from Another Planet, The (1984)",Drama|Sci-Fi
4,1853,5,"Black Hole, The (1979)",Sci-Fi


,movieId,rating,name,genre
0,1258,2.480544,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi
1,1275,2.243518,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
2,1277,2.223127,Star Trek IV: The Voyage Home (1986),Action|Adventure|Sci-Fi
3,319,2.051509,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
4,1273,1.965524,Star Trek VI: The Undiscovered Country (1991),Action|Adventure|Sci-Fi


Seems like it is actually working)))

### NCF

https://www.kaggle.com/code/jamesloy/deep-learning-based-recommender-systems

In [17]:
df_ratings["rank_latest"] = df_ratings.groupby(["userId"])["timestamp"].rank(
    method="first", ascending=False
)

train_ratings = df_ratings[df_ratings["rank_latest"] > 2]
val_ratings = df_ratings[df_ratings["rank_latest"] == 2]
test_ratings = df_ratings[df_ratings["rank_latest"] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[["userId", "movieId", "rating"]]
val_ratings = val_ratings[["userId", "movieId", "rating"]]
test_ratings = test_ratings[["userId", "movieId", "rating"]]

In [18]:
train_ratings.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [19]:
"rating" in train_ratings.columns

True

In [20]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training

    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds

    """

    def __init__(self, ratings):
        if "rating" in ratings.columns:
            # train data
            self.userIds, self.movieIds, self.ratings = (
                torch.tensor(ratings["userId"].values).to(torch.int),
                torch.tensor(ratings["movieId"].values).to(torch.int),
                torch.tensor(ratings["rating"].values).to(torch.float32),
            )
        else:
            # test data
            self.userIds, self.movieIds, self.ratings = (
                torch.tensor(ratings["userId"].values).to(torch.int),
                torch.tensor(ratings["movieId"].values).to(torch.int),
            )
            self.ratings = torch.empty(len(self.userIds))

    def __len__(self):
        return len(self.userIds)

    def __getitem__(self, idx):
        return self.userIds[idx], self.movieIds[idx], self.ratings[idx]

In [21]:
class NCF_internal(nn.Module):
    """Neural Collaborative Filtering (NCF)

    Args:
        num_users (int): Number of unique users
        num_items (int): Number of unique items
        ratings (pd.DataFrame): Dataframe containing the movie ratings for training
        all_movieIds (list): List containing all movieIds (train + test)
    """

    def __init__(self, num_users, num_items):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)

    def forward(self, user_input, item_input):

        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.ReLU6()(self.output(vector))

        return pred.flatten()

    # def training_step(self, batch, batch_idx):
    #     user_input, item_input, labels = batch
    #     predicted_labels = self(user_input, item_input)
    #     loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
    #     return loss

    # def configure_optimizers(self):
    #     return torch.optim.Adam(self.parameters())

    # def train_dataloader(self):
    #     return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
    #                       batch_size=512, num_workers=4)

In [70]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items):
        super().__init__()
        # configure the model
        self.model = NCF_internal(num_users, num_items)
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=3e-4)
        self.device = "cpu"

    def to(self, device):
        super().to(device)
        self.device = device

    def train(self, train_ratings, val_ratings=None, n_epochs: int = 5):
        # configure train data loader
        train_loader = DataLoader(MovieLensTrainDataset(train_ratings), batch_size=512)
        batch_size = 512
        num_iterations = len(train_loader)

        # configure val data loader
        flag_validation = False
        if val_ratings is not None:
            val_loader = DataLoader(MovieLensTrainDataset(val_ratings), batch_size=512)
            flag_validation = True

        for epoch in tqdm(range(n_epochs), desc="Epochs"):

            # train
            total_train_loss = 0
            self.model.train()
            for userIds, movieIds, ratings in tqdm(train_loader, desc="Training"):

                pred_train = self.model(
                    userIds.to(self.device), movieIds.to(self.device)
                )
                loss_train = self.loss_fn(pred_train, ratings.to(self.device))

                self.optimizer.zero_grad()
                loss_train.backward()
                self.optimizer.step()
                total_train_loss += loss_train.item()

            print("Epoch:", epoch)
            print("Train loss", round(total_train_loss / num_iterations, 5))

            if flag_validation:
                self.model.eval()
                val_preds = torch.empty(0)
                val_ratings = torch.empty(0)
                with torch.no_grad():
                    for userIds, movieIds, ratings in val_loader:

                        # training step
                        pred = self.model(
                            userIds.to(self.device), movieIds.to(self.device)
                        )
                        val_preds = torch.cat([val_preds, pred.cpu()])
                        val_ratings = torch.cat([val_ratings, ratings])

                loss_val = self.loss_fn(val_preds, val_ratings).item()
                print("Val loss", round(loss_val, 5))

            print("-" * 50)

    def predict_straight(self, test_ratings):
        # configure test data loader
        test_loader = DataLoader(MovieLensTrainDataset(test_ratings), batch_size=512)

        self.model.eval()
        test_preds = torch.empty(0)
        with torch.no_grad():
            for userIds, movieIds, _ in tqdm(test_loader):

                # training step
                pred = self.model(userIds.to(self.device), movieIds.to(self.device))
                test_preds = torch.cat([test_preds, pred.cpu()])

        return test_preds

    # def predict(self, userId):
    #     # predict all the ratings for the concrete user

In [71]:
device = "cpu"

In [72]:
ncf = NCF(df_ratings.userId.nunique(), df_ratings.movieId.nunique())
ncf.to(device)

In [73]:
ncf.train(train_ratings, val_ratings, n_epochs=50)

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/1930 [00:00<?, ?it/s]

Epoch: 0
Train loss 1.99008
Val loss 1.34709
--------------------------------------------------


Training:   0%|          | 0/1930 [00:00<?, ?it/s]

Epoch: 1
Train loss 1.20738
Val loss 1.21664
--------------------------------------------------


...

Epoch: 47
Train loss 0.83004
Val loss 0.95136
--------------------------------------------------


Training:   0%|          | 0/1930 [00:00<?, ?it/s]

Epoch: 48
Train loss 0.8277
Val loss 0.94989
--------------------------------------------------


Training:   0%|          | 0/1930 [00:00<?, ?it/s]

Epoch: 49
Train loss 0.8254
Val loss 0.94903
--------------------------------------------------


In [25]:
ncf.predict_straight(test_ratings)

  0%|          | 0/12 [00:00<?, ?it/s]

tensor([2.9348, 3.0938, 3.7552,  ..., 3.6536, 4.5211, 4.3658])

In [69]:
ncf.predict_straight(test_ratings)

  0%|          | 0/12 [00:00<?, ?it/s]

tensor([3.2317, 2.9274, 3.7919,  ..., 3.3790, 5.1271, 4.0765])

In [74]:
ncf.predict_straight(test_ratings)

  0%|          | 0/12 [00:00<?, ?it/s]

tensor([3.4524, 2.6045, 3.9055,  ..., 4.0072, 4.3311, 3.9072])

In [26]:
test_ratings

,userId,movieId,rating
25,0,47,5
66,1,1550,3
232,2,1900,4
235,3,2743,4
258,4,279,2
...,...,...,...
998803,6035,2602,1
999557,6036,907,4
999731,6037,1094,5
999764,6038,851,4


In [27]:
len(train_loader)

NameError: name 'train_loader' is not defined

In [28]:
for batch in tqdm(train_loader, desc="Iterations"):
    print(batch[0])
    break

NameError: name 'train_loader' is not defined